In [ ]:
# Databricks notebook source
# =============================================================================
# 🚀 UAT STAGING PROMOTION - NEW WORKFLOW (CONFIG-DRIVEN)
# =============================================================================
# Purpose: Promote registered model from model_registration.py to Staging alias
# Prerequisites: Run data_preparation.py → train.py → model_registration.py
# Compatible with new workflow
# =============================================================================

import mlflow
from mlflow.tracking import MlflowClient
import time
import yaml
import sys
import traceback
from typing import Optional, Dict
from datetime import datetime

print("=" * 80)
print("🚀 UAT STAGING PROMOTION (NO SLACK VERSION)")
print("=" * 80)

# =============================================================================
# ✅ LOAD PIPELINE CONFIGURATION
# =============================================================================
print("\n📋 Step 1: Loading configuration from pipeline_config.yml...")

try:
    with open("pipeline_config.yml", "r") as f:
        pipeline_cfg = yaml.safe_load(f)
    
    print(f"✅ Configuration loaded successfully!")
    
except FileNotFoundError:
    print("❌ ERROR: pipeline_config.yml not found!")
    sys.exit(1)
except Exception as e:
    print(f"❌ ERROR loading configuration: {e}")
    traceback.print_exc()
    sys.exit(1)

# =============================================================================
# ✅ CONFIGURATION CLASS (Slack Removed)
# =============================================================================
class Config:
    """Configuration manager - reads from pipeline_config.yml"""
    
    def __init__(self):
        MODEL_TYPE = pipeline_cfg["model"]["type"]
        UC_CATALOG = pipeline_cfg["model"]["catalog"]
        UC_SCHEMA = pipeline_cfg["model"]["schema"]
        BASE_NAME = pipeline_cfg["model"]["base_name"]
        
        self.MODEL_NAME = f"{UC_CATALOG}.{UC_SCHEMA}.{BASE_NAME}_{MODEL_TYPE}"
        self.MODEL_TYPE = MODEL_TYPE

        self.STAGING_ALIAS = pipeline_cfg["aliases"]["staging"]
        self.PRODUCTION_ALIAS = pipeline_cfg["aliases"]["production"]
        self.BEST_ALIAS = pipeline_cfg["aliases"]["best"]
        
        self.PRIMARY_METRIC = pipeline_cfg["metrics"]["classification"]["primary_metric"]
        self.DIRECTION = pipeline_cfg["metrics"]["classification"]["direction"]
        self.TOLERANCE = 1e-6
        
        print(f"\n📊 Configuration Summary:")
        print(f"   Model Name: {self.MODEL_NAME}")
        print(f"   Alias: @{self.STAGING_ALIAS}")
        print(f"   Metric: {self.PRIMARY_METRIC} ({self.DIRECTION})")

# Initialize config
config = Config()

print("=" * 80)

# =============================================================================
# ✅ INITIALIZE MLFLOW
# =============================================================================
print("\n🔧 Step 2: Initializing MLflow...")

try:
    mlflow.set_tracking_uri("databricks")
    mlflow.set_registry_uri("databricks-uc")
    client = MlflowClient()
    
    print("✅ MLflow initialized successfully")

except Exception as e:
    print(f"❌ Failed to initialize MLflow: {e}")
    sys.exit(1)

print("\n📢 Staging Promotion Pipeline Started")

# =============================================================================
# 🔧 HELPER FUNCTIONS
# =============================================================================

def wait_until_ready(version: int, timeout: int = 300) -> bool:
    print(f"\n⏳ Waiting for model v{version} to become READY...")

    start = time.time()
    while time.time() - start < timeout:
        mv = client.get_model_version(config.MODEL_NAME, version)
        if mv.status == "READY":
            print(f"   ✅ Model v{version} READY")
            return True
        elif mv.status == "FAILED_REGISTRATION":
            print(f"   ❌ Model registration failed")
            return False
        time.sleep(5)

    print(f"   ⏰ Timeout waiting for READY")
    return False


def get_metric_from_run(run_id: str) -> Optional[float]:
    try:
        run = client.get_run(run_id)
        metric_value = run.data.metrics.get(config.PRIMARY_METRIC)
        print(f"   ↳ Metric = {metric_value}")
        return metric_value
    except:
        return None


def get_latest_registered_version():
    print("\n📋 Fetching latest registered model version...")

    versions = list(client.search_model_versions(f"name='{config.MODEL_NAME}'"))
    if not versions:
        print("❌ No model versions found")
        return None

    latest = sorted(versions, key=lambda m: int(m.version), reverse=True)[0]
    
    return {
        'version': int(latest.version),
        'run_id': latest.run_id,
        'metric': get_metric_from_run(latest.run_id)
    }


def get_current_staging_version():
    print("\n📋 Checking current staging model...")

    try:
        staging = client.get_model_version_by_alias(config.MODEL_NAME, config.STAGING_ALIAS)
        return {
            'version': int(staging.version),
            'run_id': staging.run_id,
            'metric': get_metric_from_run(staging.run_id)
        }
    except:
        print("ℹ️ No staging model exists yet")
        return None


def should_promote(new, staging) -> tuple:
    print("\n📋 Comparing performance...")

    if staging is None:
        return True, "First model promotion"

    if new["metric"] is None:
        return True, "No metric available for comparison"

    if abs(new["metric"] - staging["metric"]) <= config.TOLERANCE:
        return False, "No improvement (metric equal)"

    if config.DIRECTION == "maximize" and new["metric"] > staging["metric"]:
        return True, "Improved performance"

    if config.DIRECTION == "minimize" and new["metric"] < staging["metric"]:
        return True, "Lower is better"

    return False, "Performance worse"


def promote_to_staging(version: int, reason: str):
    print(f"\n🚀 Promoting v{version} → @{config.STAGING_ALIAS}")

    if not wait_until_ready(version):
        print("❌ Promotion failed (model not ready)")
        return False

    client.set_registered_model_alias(config.MODEL_NAME, config.STAGING_ALIAS, version)

    print(f"✅ Model promoted successfully (Reason: {reason})")
    return True


# =============================================================================
# 🎬 MAIN EXECUTION
# =============================================================================

def main():

    latest = get_latest_registered_version()
    staging = get_current_staging_version()

    promote_flag, reason = should_promote(latest, staging)

    if promote_flag:
        promote_to_staging(latest['version'], reason)
    else:
        print(f"\n⚠️ Promotion skipped → {reason}")

    print("\n🎉 Staging Promotion process completed!")


if __name__ == "__main__":
    main()
